In [ ]:
from imutils import face_utils
import datetime
import imutils
import time
import dlib
import cv2, math
import numpy as np
from imutils import face_utils, rotate_bound


# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
print("[INFO] loading facial landmark predictor...")
model = "shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(model) # link to model: http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2

video_capture = cv2.VideoCapture("http://192.168.0.102:8080/video")
cv2.imshow('Video', np.empty((5,5),dtype=float))

#points are tuples in the form (x,y)
# returns angle between points in degrees
def calculate_inclination(point1, point2):
    x1,x2,y1,y2 = point1[0], point2[0], point1[1], point2[1]
    incl = -180/math.pi*math.atan((float(y2-y1))/(x2-x1))
    return incl

def calculate_boundbox(list_coordinates):
    x = min(list_coordinates[:,0])
    y = min(list_coordinates[:,1])
    w = max(list_coordinates[:,0]) - x
    h = max(list_coordinates[:,1]) - y
    return (x,y,w,h)

def get_face_boundbox(points, face_part):
    if face_part == 1:
        (x,y,w,h) = calculate_boundbox(points[17:22]) #left eyebrow
    elif face_part == 2:
        (x,y,w,h) = calculate_boundbox(points[22:27]) #right eyebrow
    elif face_part == 3:
        (x,y,w,h) = calculate_boundbox(points[36:42]) #left eye
    elif face_part == 4:
        (x,y,w,h) = calculate_boundbox(points[42:48]) #right eye
    elif face_part == 5:
        (x,y,w,h) = calculate_boundbox(points[29:36]) #nose
    elif face_part == 6:
        (x,y,w,h) = calculate_boundbox(points[48:68]) #mouth
    return (x,y,w,h)

while cv2.getWindowProperty('Video', 0) >= 0:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # detect faces in the grayscale frame
    rects = detector(gray, 0)

    # loop over the face detections
    for rect in rects:
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)

        for i in range(1,7):
            (x,y,w,h) = get_face_boundbox(shape, i)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 1)

        incl = calculate_inclination(shape[17], shape[26])

        img = cv2.imread("nose.png")
        rows,cols = img.shape[0], img.shape[1]
        M = cv2.getRotationMatrix2D((cols/2,rows/2),incl,1)
        dst = cv2.warpAffine(img,M,(cols,rows))
        dst = rotate_bound(img, incl)
        cv2.imshow('sprite',dst)

        print("Pixels distance points in mouth: ", shape[66][1] - shape[62][1])

        x,y, w, h = rect.left(), rect.top(), rect.width(), rect.height()

        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)

        # loop over the (x, y)-coordinates for the facial landmarks
        # and draw them on the image
        for (x, y) in shape:
            cv2.circle(frame, (x, y), 1, (0, 0, 255), -1)

    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()

In [ ]:
#this filter puts heart emojis on your eyes when you open you mouth

import dlib
import numpy as np
import cv2

cap = cv2.VideoCapture("http://192.168.0.102:8080/video")
img = cv2.imread("nose.png")

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat") 

while True:
    _, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #convert the video into gray for faster processing
    
    faces = detector(gray) #detect faces in the gray frame 
    for face in faces:
        
        a1 = face.left()
        b1 = face.top()
        a2 = face.right()
        b2 = face.bottom()
        faceHeight = b2 - b1
        faceWidth = a2- a1
        #print(faceHeight,faceWidth)
        #cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 3)
        
        landmarks = predictor(gray, face)
        
        leftEyeLeft = (landmarks.part(36).x,landmarks.part(36).y)
        leftEyeRight = (landmarks.part(39).x,landmarks.part(39).y)
        rightEyeLeft = (landmarks.part(42).x,landmarks.part(42).y)
        rightEyeRight = (landmarks.part(45).x,landmarks.part(45).y)
        
        heartWidth = leftEyeRight[0] - leftEyeLeft[0]
        heartWidth = 3 * heartWidth
        heartHeight = heartWidth
        
        heartImage = cv2.resize(img, (heartWidth,heartHeight))
        heartImageGray = cv2.cvtColor(heartImage, cv2.COLOR_BGR2GRAY)
        
        leftEyeCenter = ((leftEyeLeft[0] + leftEyeRight[0])/2,(leftEyeLeft[1] + leftEyeRight[1])/2)
        leftTopLeft = (int(leftEyeCenter[0] - heartWidth / 2),int(leftEyeCenter[1] - heartHeight / 2))
        leftBottomRight = (int(leftEyeCenter[0] + heartWidth / 2),int(leftEyeCenter[1] + heartHeight / 2))
        
        rightEyeCenter = ((rightEyeLeft[0] + rightEyeRight[0])/2,(rightEyeLeft[1] + rightEyeRight[1])/2)
        rightTopLeft = (int(rightEyeCenter[0] - heartWidth / 2),int(rightEyeCenter[1] - heartHeight / 2))
        rightBottomRight = (int(rightEyeCenter[0] + heartWidth / 2),int(rightEyeCenter[1] + heartHeight / 2))        
        
        x1 = landmarks.part(62).x
        y1 = landmarks.part(62).y
        x2 = landmarks.part(66).x
        y2 = landmarks.part(66).y
        
        if (y2-y1)/faceHeight > 0.05:
            print("mouth open")
            heartAreaLeft = frame[leftTopLeft[1] : leftTopLeft[1] + heartHeight,
                              leftTopLeft[0] : leftTopLeft[0] + heartWidth]  
            _, heartMask = cv2.threshold(heartImageGray, 25, 255, cv2.THRESH_BINARY_INV)
            heartAreaNoHeartLeft = cv2.bitwise_and(heartAreaLeft,heartAreaLeft, mask = heartMask)
            finalHeartLeft = cv2.add(heartAreaNoHeartLeft,heartImage)
            frame[leftTopLeft[1] : leftTopLeft[1] + heartHeight,
                         leftTopLeft[0] : leftTopLeft[0] + heartWidth]  = finalHeartLeft
                  
            heartAreaRight = frame[rightTopLeft[1] : rightTopLeft[1] + heartHeight,
                              rightTopLeft[0] : rightTopLeft[0] + heartWidth]  
            heartAreaNoHeartRight = cv2.bitwise_and(heartAreaRight,heartAreaRight, mask = heartMask)
            finalHeartRight = cv2.add(heartAreaNoHeartRight,heartImage)
            frame[rightTopLeft[1] : rightTopLeft[1] + heartHeight,
                         rightTopLeft[0] : rightTopLeft[0] + heartWidth]  = finalHeartRight
                  
        
    cv2.imshow("frame", frame)
    
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break
        cap.release()
        cv2.destroyAllWindows()
        break


In [3]:
import dlib
import numpy as np
import cv2

cap = cv2.VideoCapture("http://192.168.0.102:8080/video")
img = cv2.imread("heart.png")

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat") 

while True:
    _, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #convert the video into gray for faster processing
    
    faces = detector(gray) #detect faces in the gray frame 
    for face in faces:
        
        a1 = face.left()
        b1 = face.top()
        a2 = face.right()
        b2 = face.bottom()
        faceHeight = b2 - b1
        faceWidth = a2- a1
        print(a1)
        print(a2)
        faceHeight = b2 - b1
        faceWidth = a2- a1
        #print(faceHeight,faceWidth)
        #cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 3)
        
        landmarks = predictor(gray, face)
        
        leftEyeLeft = (landmarks.part(36).x,landmarks.part(36).y)
        leftEyeRight = (landmarks.part(39).x,landmarks.part(39).y)
        rightEyeLeft = (landmarks.part(42).x,landmarks.part(42).y)
        rightEyeRight = (landmarks.part(45).x,landmarks.part(45).y)
        
        heartWidth = leftEyeRight[0] - leftEyeLeft[0]
        heartWidth = 3 * heartWidth
        heartHeight = heartWidth
leftEyeLeft

463
998
463
998
463
998
463
998
463
998
584
1029
584
1029
584
1029


KeyboardInterrupt: 

In [4]:
leftEyeLeft

(679, 549)

In [6]:
leftEyeRight[0] - leftEyeLeft[0]

53